In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
df_all = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')
matches_list1 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_01')
matches_list2 = pd.read_excel('/Users/roumyadas/Desktop/IPL_Simulation/Fixtures/IPL_2024_schedule.xlsx',
                            sheet_name='Season_02')

matches_list1['match_id'] = matches_list1['num'].apply(lambda x: 'M00' + str(x) if x<=9 else 'M0' + str(x))
matches_list2['match_id'] = matches_list2['num'].apply(lambda x: 'S02M00' + str(x) if x<=9 else 'S02M0' + str(x))

matches_list = pd.concat([matches_list1[['num','Spin','match_id']],matches_list2[['num','Spin','match_id']]],
                         axis=0)
df_all = df_all.merge(matches_list[['match_id','Spin']], on='match_id', how='left')
df_all['pitch'] = df_all['Spin'].apply(lambda x: 'pace' if x<=40 
                                       else 'spin' if x>=60
                                      else 'neutralish')

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
#df_all = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_02/df_all_round_sim.csv')
m_m = df_all.match_id.unique()[-1]
#m_m = 'S02M065'
df = df_all[df_all.match_id==m_m]

In [13]:
path__ = '/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/venue_stats_indi.xlsx'

existing_venue_file = pd.read_excel(path__)


In [15]:
sub_df = df
bat1 = sub_df[sub_df.innings==1]['runs_scored'].max()
wkt1 = sub_df[sub_df.innings==1]['wickets_down'].max()
bat2 = sub_df[sub_df.innings==2]['runs_scored'].max()
wkt2 = sub_df[sub_df.innings==2]['wickets_down'].max()
venue = sub_df['venue'].unique()[0]
pitch = sub_df['pitch'].unique()[0]
t1 = sub_df[sub_df.innings==1]['batting_team'].unique()[0]
t2 = sub_df[sub_df.innings==2]['batting_team'].unique()[0]
winner = (t1 if bat1>bat2 else t2)
bat_1st_win = (1 if bat1>bat2 else 0)
bat_2nd_win = (1 if bat1<bat2 else 0)

row_ = [m_m,venue,bat1,wkt1,t1,t2,bat2,wkt2,pitch,bat_1st_win,bat_2nd_win,winner]
row_df = pd.DataFrame([row_], columns=existing_venue_file.columns)

concatted_df = pd.concat([existing_venue_file, row_df], ignore_index=True)

In [23]:
concatted_df.columns

Index(['match_id', 'venue', '1st_innings_total', '1st_innings_wkts',
       '1st_bat_team', '2nd_bat_team', '2nd_innings_total', '2nd_innings_wkts',
       'pitch', 'bat_1st_win', 'bat_2nd_win', 'winner'],
      dtype='object')

In [17]:
concatted_df.to_excel(path__, index=None)


## calculations

##### venue-wise bat 1st avg score, pitch-wise




In [37]:
bat1_pitch = concatted_df.drop_duplicates().groupby(['venue','pitch'])\
                [['1st_innings_total','1st_innings_wkts']].mean().reset_index()

bat1_pitch = bat1_pitch.round(1)

bat1_pitch['score'] = bat1_pitch[['1st_innings_total','1st_innings_wkts']].apply(lambda x: f"{x[0]}-{x[1]}", 
                                                                           axis=1)


bat1_ov = concatted_df.drop_duplicates().groupby(['venue'])\
                [['1st_innings_total','1st_innings_wkts','bat_1st_win']].mean().reset_index()

bat1_ov[['1st_innings_total','1st_innings_wkts']] = bat1_ov[['1st_innings_total','1st_innings_wkts']].round(1)

bat1_ov['score'] = bat1_ov[['1st_innings_total','1st_innings_wkts']].apply(lambda x: f"{x[0]}-{x[1]}", 
                                                                           axis=1)
bat1_ov['win%'] = bat1_ov['bat_1st_win'].round(4) 
bat1_ov['win%'] = bat1_ov['win%']*100

In [39]:
excel_filename = "/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/venue_stats.xlsx"

# Dictionary mapping sheet names to DataFrames
sheets_data = {
    'pitch_wise': bat1_pitch,
    'overall': bat1_ov
    
}

with pd.ExcelWriter(excel_filename, engine='xlsxwriter') as writer:
    # Writing all sheets
    for sheet_name, df in sheets_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Get workbook
    workbook = writer.book  

    # Define formats
    bold_format = workbook.add_format({'bold': True})  # Bold text
    large_font_format = workbook.add_format({'font_size': 14})  # Increased font size
    red_text_format = workbook.add_format({'font_color': 'red'})  # Red text

    # Apply formatting to each sheet
    for sheet_name, df in sheets_data.items():
        worksheet = writer.sheets[sheet_name]
        
        # 1️⃣ Bold all column names (header row)
        for col_idx, col_name in enumerate(df.columns):
            worksheet.write(0, col_idx, col_name, bold_format)
        
        # 3️⃣ Bold entire column 1 (index 0) of every sheet
        worksheet.set_column(0, 0, None, bold_format)

        
        # 6️⃣ Adjust column widths dynamically
        for col_idx, col_name in enumerate(df.columns):
            max_length = max(df[col_name].astype(str).apply(len).max(), len(col_name))  
            # Max of column values & header
            worksheet.set_column(col_idx, col_idx, max_length + 2)  # Adding padding for readability
            
        # 7️⃣ Freeze the first row of every sheet
        worksheet.freeze_panes(1, 0)
        
        # 8️⃣ **Add Filter to all columns**
        worksheet.autofilter(0, 0, 0, len(df.columns) - 1)  # Apply filter to the first row across all columns


print(f"Excel file '{excel_filename}' created successfully with formatting!")


Excel file '/Users/roumyadas/Desktop/IPL_Simulation/Season_02/Stats/venue_stats.xlsx' created successfully with formatting!
